In [10]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForCausalLM
import torch
import re

In [3]:
# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained("./saved_model_ver2")

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="./saved_model_ver2",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
# Перемещение модели на GPU или CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [62]:
def inference(category, rating):
    # Формирование входного текста
    input_text = f"Category: {category}.\nRating: {rating}.\nReview:"
    # Токенизация текста
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    output = model.generate(
        **inputs,
        max_length=60,
        temperature=0.9,
        no_repeat_ngram_size=2,
        do_sample=True,
        num_return_sequences=3,
        eos_token_id=tokenizer.eos_token_id,  # Установка токена конца последовательности
        pad_token_id=tokenizer.eos_token_id,  # Использовать тот же токен для паддинга
        early_stopping=False,  # Раннее завершение на токене конца
        top_k=30,         # Уменьшаем количество рассматриваемых верхних k слов
        top_p=0.8,        # Уменьшаем "ядерность" распределения
    )
    # Декодирование всех сгенерированных текстов
    for i, sequence in enumerate(output):
        generated_text = tokenizer.decode(sequence, skip_special_tokens=True)
        print(f"Generated text {i + 1}:")
        print(generated_text)
        print()
        # raw_text = re.findall(r'(?<=Review:  ).*', generated_text)[0]
        # print(f'Raw text - {raw_text}\n')

# Примеры инференса

In [80]:
inference('Магазин детских игрушек', 5)

Generated text 1:
Category: Магазин детских игрушек.
Rating: 5.
Review: большой выбор игрушек и игрушек для деток с рождения и до 3 лет игрушки можно найти в нашем магазине очень большой ассортимент игрушек разных возрастов и размеров и по цене и на любой возраст  в частности есть

Generated text 2:
Category: Магазин детских игрушек.
Rating: 5.
Review: очень уютный магазин с большой территорией и с большим выбором товаров и игрушек и аксессуаров для детей и взрослых большой выбор товара для мальчиков и девочек из разных уголков нашей страны а также детская мебель для самых маленьких

Generated text 3:
Category: Магазин детских игрушек.
Rating: 5.
Review: магазин с большим ассортиментом товаров для детей и родителей  из минусов  большая очередь в кассах и очень часто приходится стоять в очереди и ждать пока освободится касса с товаром в основном в это время



In [65]:
inference('Магазин автозапчастей', 4)

Generated text 1:
Category: Магазин автозапчастей.
Rating: 4.
Review: неплохой магазин автотоваров  большой ассортимент товара есть на любой вкус и кошелк есть все необходимое есть и запчасти и автоаксессуары для автотюнинг автомобилей и автомобилей с пробегом есть

Generated text 2:
Category: Магазин автозапчастей.
Rating: 4.
Review: все хорошо если есть свободные места то можно было бы еще и в этот раз сделать побольше  в этом году все отлично  я купила магнитолу и магнитола для телефона  с магнитолой

Generated text 3:
Category: Магазин автозапчастей.
Rating: 4.
Review: в целом нормально но в последнее время цены на сайте стали заметно выше чем в других магазинах в городе с ценами на некоторые позиции я бы не советовал обращаться в этот магазин по данному адресу в итоге мне



In [71]:
inference('Поликлиника', 3)

Generated text 1:
Category: Поликлиника.
Rating: 3.
Review: поликлиника работает с 2000 года по настоящее время но сейчас в поликлинике постоянно чтото не работает в связи с ухудшением качества оказания медицинской помощи и отсутствием новых врачей и не знаю как в других клиниках но

Generated text 2:
Category: Поликлиника.
Rating: 3.
Review: все хорошо но есть один маленький нюанс в том что с поликлиникой не работают только терапевты и педиатры и терапевт у них очередь в регистратуру и не все врачи приходят на прием к терапев

Generated text 3:
Category: Поликлиника.
Rating: 3.
Review: все хорошо но есть одно но очень грустное  это  когда я приехала к врачу в регистратуру на дом по записи  мне сказали что у меня есть запись  я не смогла записаться на прием 



In [81]:
inference('Кафе', 2)

Generated text 1:
Category: Кафе.
Rating: 2.
Review: очень долгое обслуживание и очень долго обслуживание заказывал суши на кассе но так и не дождался заказ принесли в 1530 минут и в итоге на вопрос о готовности заказа я ответил что готов уже был в 2030 так

Generated text 2:
Category: Кафе.
Rating: 2.
Review: в целом заведение хорошее но есть один нюанс очень долго готовят еду не всегда получается есть очень маленькие порции поэтому если в меню есть чтото необходимое то лучше подождать пока вы подойдте к выбору в итоге вы увидите

Generated text 3:
Category: Кафе.
Rating: 2.
Review: очень понравился этот ресторан на улице у метро пр-т мира  есть и минусы но в целом еда очень вкусная в кафе  не вкусное меню с сыром и колбасой и без  это не еда а



In [79]:
inference('Гостиница', 1)

Generated text 1:
Category: Гостиница.
Rating: 1.
Review: очень разочаровал отель в первую очередь тем что он находится в самом центре города и поэтому мы жили на 1 этаже в отеле с видом на город а точнее на набережную и с одной стороны набережной в целом впечатление от

Generated text 2:
Category: Гостиница.
Rating: 1.
Review: отличный отель с шикарным видом на море и море но есть одно но в номере не очень уютно  туалет не работал  с этим все в порядке  персонал  доброжелательный не вежливый в номерах есть чайник

Generated text 3:
Category: Гостиница.
Rating: 1.
Review: отличный отель в центре города рядом с набережной рядом со станцией метро  в номере на первом этаже есть все необходимое для комфортного проживания в гостинице есть холодильник телевизор холодильник посуда чай кофе набор посуды набор полотенец шампунь

